In [ ]:
# VERSION 1.1 2024-05-26
# This script 
# 1. prompts user for two frequencies in Hz
# 2. determines the interval between those frequencies in Cents
# 3. determines the interval between those frequencies in semitones ±50 Cts.
# 4. determines the note pitch ±50 Ct. of the two frequencies relative to concert pitch A4. 


# function to prompt user input for two valid frequencies 
def get_frequency_input (prompt):
   
    while True: # loops the input request until a valid input is received
        try:
            # request user input
            user_input = input(prompt)
            # try to convert the input to a float
            value = float(user_input)
            # check that the value is positive
            if value > 0:
                return value
            else: # if value is ≤0, inform user and prompt again
                print("Frequency must be a number greater than 0.")
        except ValueError:
            # if conversion to float fails, inform user and prompt again
            print("Frequency must be a number greater than 0.")



# this is the actual function to calculate the interval
def calc_interval():
    
    import numpy as np #needed for log2 conversion of frequency -> cents
    
    # Request user input for freq1 and freq2 by calling the validated input function
    freq1 = get_frequency_input("Enter the first frequency in Hz: ")
    freq2 = get_frequency_input("Enter the second frequency in Hz: ")
    print() # adds empty line break
        
    # figure out which is the larger and which the smaller frequency
    if freq1 > freq2:
        larger_freq = freq1
        smaller_freq = freq2
    else:
        larger_freq = freq2
        smaller_freq = freq1
        
    # calculate the interval in cents using numpy
    intvl_cents = 1200 * np.log2(larger_freq / smaller_freq)
        
    # Print the result in cents
    print(f"The interval between {freq1} Hz and {freq2} Hz is {intvl_cents:.2f} cents")
     # ".2" rounds to two decimal places, "f" formats the display as fixed-point (not floating) decimal number

    # determine the interval in semitones
    semitones = int(intvl_cents / 100) # use 'int' because 'round' will round .5 to nearest even number
    # determine the remaining cents
    remaining_cents = intvl_cents - (semitones * 100)
        
    # for remaining cents >50, add 1 semitone and subtract 100 to turn remaining cents into negative value
    if remaining_cents > 50:
        semitones += 1
        remaining_cents -= 100
    else: 
        pass 
         
    # print the result
    if remaining_cents == 0:
        # correct output for grammar if only 1 semitone
        if semitones == 1: 
            print(f"The interval is {semitones} semitone exactly.") # singular version
        else: 
            print(f"The interval is {semitones} semitones exactly.") # plural version
    else: 
        # correct output for grammar if only 1 semitone
        if semitones == 1: 
            print(f"The interval is {semitones} semitone {remaining_cents:+.2f} cents.") # singular version
        else: 
            print(f"The interval is {semitones} semitones {remaining_cents:+.2f} cents.") # plural version
            # "+" ensures display of +/- sign, ".2" rounds to two decimal places, 
            # "f" formats as fixed-point (not floating) decimal number
    print()
    
#################################################################################################

    # function to determine the note closest to given frequency, calculated UPWARDS from next-lower 'A'
    def determine_note(user_freq,reference_freq):
        octave_count = 4 # determines reference_freq as 'A4' (scientific pitch)
        current_freq = reference_freq
        if user_freq < reference_freq:
            # determine the base note 'A' from which to calculate interval to note
            while current_freq > user_freq: # keep going down from A4 in octaves until base A is lower than user's note
                current_freq /= 2 # octave down
                octave_count -= 1 # increment octave counter downwards from 'A4' to 'A3', 'A2', etc. 
#            return current_freq, octave_count

        else:
            if user_freq == reference_freq: # if user note is exactly concert pitch, do nothing
                pass
#            return current_freq, octave_count
            else:
                while current_freq <= user_freq: # keep going up from A4 in Octaves until base 'A' is higher than user's note
                    current_freq *= 2 # octave up
                    octave_count += 1 # increment octave counter upwards from 'A4' to 'A5', 'A6', etc.
            if current_freq > user_freq: # correct back down to base 'A' after overshooting   
                current_freq /= 2
                octave_count -= 1
#            return current_freq, octave_count

        # calculate the interval in cents using numpy
        intvl_cents_absolute_lower = 1200 * np.log2(user_freq / current_freq)
        return intvl_cents_absolute_lower, octave_count

#################################################################################################

    
    ref_pitch = float(440) # define concert pitch at A4 in Hz

    print (f"Concert pitch is set to A4 = {ref_pitch:.2f} Hz")
    print ()

    # define list of notes, adding extra "A" for intervals > 11 semitones +50 Cts (which convert to 12 - <50Cts).
    notes = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A"]
    
    # determine note pitch of LOW FREQUENCY
    intvl_cents_absolute_low, octave_count_low = determine_note(smaller_freq,ref_pitch)
    print (f"The lower note is {intvl_cents_absolute_low:.2f} cents above A{octave_count_low} at A4 = {ref_pitch:.2f} Hz")

    
    # determine the interval of lower note to baseline 'A' in semitones
    semitones_low = int(intvl_cents_absolute_low / 100)
    # determine the remaining cents
    remaining_cents_low = intvl_cents_absolute_low - (int(semitones_low) * 100)
        
    # for remaining cents >50, add 1 semitone and subtract 100 to turn remaining cents into negative value
    if remaining_cents_low > 50:
        semitones_low += 1
        remaining_cents_low -= 100
    else: 
        pass 

    # define "octave name" with correct octave count for lower note for notes from 'C' to 'G#'
    if semitones_low > 2:
        octave_name_low = octave_count_low + 1
    else:
        octave_name_low = octave_count_low
    
    # print the result
    if remaining_cents_low == 0:
        # correct output for grammar if only 1 semitone
        if semitones_low == 1: 
            print(f"The lower note is exactly {semitones_low} semitone above A{octave_count_low} at A4 = {ref_pitch:.2f} Hz.") # singular version
        else: 
            print(f"The lower note is exactly {semitones_low} semitones above A{octave_count_low} at A4 = {ref_pitch:.2f} Hz.") # plural version

        print(f"The lower note is a {notes[semitones_low]}{octave_name_low} at A4 = {ref_pitch:.2f} Hz.")
    else: 
        # correct output for grammar if only 1 semitone
        if semitones_low == 1: 
            print(f"The lower note is {semitones_low} semitone {remaining_cents_low:+.2f} cents above A{octave_count_low} at A4 = {ref_pitch:.2f} Hz.") # singular version
        else: 
            print(f"The lower note is {semitones_low} semitones {remaining_cents_low:+.2f} cents above A{octave_count_low} at A4 = {ref_pitch:.2f} Hz.") # plural version
            
        print(f"The lower note is a {notes[semitones_low]}{octave_name_low} {remaining_cents_low:+.2f} cents at A4 = {ref_pitch:.2f} Hz.")
            # "+" ensures display of +/- sign, ".2" rounds to two decimal places, 
            # "f" formats as fixed-point (not floating) decimal number

    print()

    # determine note pitch of UPPER FREQUENCY
    intvl_cents_absolute_hi, octave_count_hi = determine_note(larger_freq,ref_pitch)
    print (f"The upper note is {intvl_cents_absolute_hi:.2f} cents above A{octave_count_hi} at A4 = {ref_pitch:.2f} Hz")

    # determine the interval of upper note to baseline 'A' in semitones
    semitones_hi = int(intvl_cents_absolute_hi / 100)
    # determine the remaining cents
    remaining_cents_hi = intvl_cents_absolute_hi - (int(semitones_hi) * 100)
        
    # for remaining cents >50, add 1 semitone and subtract 100 to turn remaining cents into negative value
    if remaining_cents_hi > 50:
        semitones_hi += 1
        remaining_cents_hi -= 100
    else: 
        pass 

    # define "octave name" with correct octave count for upper note for notes from 'C' to 'G#'
    if semitones_hi > 2:
        octave_name_hi = octave_count_hi + 1
    else:
        octave_name_hi = octave_count_hi
          
    # print the result
    if remaining_cents_hi == 0:
        # correct output for grammar if only 1 semitone
        if semitones_hi == 1: 
            print(f"The upper note is exactly {semitones_hi} semitone above A{octave_count_hi} at A4 = {ref_pitch:.2f} Hz.") # singular version
        else: 
            print(f"The upper note is exactly {semitones_hi} semitones above A{octave_count_hi} at A4 = {ref_pitch:.2f} Hz.")

        print(f"The upper note is a {notes[semitones_hi]}{octave_name_hi} at A4 = {ref_pitch:.2f} Hz.")
    else: 
        # correct output for grammar if only 1 semitone
        if semitones_hi == 1: 
            print(f"The upper note is {semitones_hi} semitone {remaining_cents_hi:+.2f} cents above A{octave_count_hi} at A4 = {ref_pitch:.2f} Hz.")
        else: 
            print(f"The upper note is {semitones_hi} semitones {remaining_cents_hi:+.2f} cents above A{octave_count_hi} at A4 = {ref_pitch:.2f} Hz.")
            
        print(f"The upper note is a {notes[semitones_hi]}{octave_name_hi} {remaining_cents_hi:+.2f} cents at A4 = {ref_pitch:.2f} Hz.")
            # "+" ensures display of +/- sign, ".2" rounds to two decimal places, 
            # "f" formats as fixed-point (not floating) decimal number




# call the main function
if __name__ == "__main__":
    calc_interval()